In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import joblib
import os

In [9]:
forwards_salary = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/salary/cap_all.csv')
forwards_rec = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/forwards_rec.csv')

In [10]:
forwards_salary = forwards_salary.rename(columns={'CAP HIT': 'salary'})
forwards_salary = forwards_salary.drop(columns=['POS','HAND','AGE','G','A','SV%', 'TOTAL','TERM','TYPE'])
forwards_salary

,PLAYERS,TEAM,PTS,salary
0,"Matthews, Auston",TOR,56,"$13,250,000"
1,"MacKinnon, Nathan",COL,102,"$12,600,000"
2,"McDavid, Connor",EDM,85,"$12,500,000"
3,"Panarin, Artemi",NYR,69,"$11,642,857"
4,"Pettersson, Elias",VAN,39,"$11,600,000"
...,...,...,...,...
1045,"Buchinger, Michael",STL,0,"$838,333"
1046,"Mynio, Sawyer",VAN,0,"$838,333"
1047,"Krygier, Cole",LAK,0,"$837,500"
1048,"Caulfield, Judd",ANA,0,"$837,500"


In [13]:
# Define the bins and corresponding salary values
bins = [0.0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, float('inf')]
labels = [950000, 2000000, 4000000, 6000000, 8000000, 10000000, 12000000]

# Use pd.cut to bin the values and map them to the salary values
forwards_rec['salary'] = pd.cut(forwards_rec['goals'], bins=bins, labels=labels, right=False)

# Display the first few rows of the updated DataFrame
forwards_rec.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/forwards_rec_two.csv')

In [35]:
over_8000000 = forwards_rec[forwards_rec['games_played'] < 246]
over_8000000

,Unnamed: 0,name,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shots_blocked_by_player,assists,position_encoded,position,salary
0,0,A.J. Greer,1.240964,0.234940,0.078313,0.042169,0.138554,1.650602,0.030120,0.813253,...,166,518.777108,0.018795,0.019398,0.246988,0.391566,0.108434,1,F,2000000
2,3,Aaron Gagnon,1.500000,0.052632,0.078947,0.000000,0.078947,0.631579,0.026316,0.921053,...,38,502.894737,0.052632,0.052105,0.026316,0.368421,0.052632,1,F,950000
3,7,Aaron Palushaj,1.220588,0.191176,0.044118,0.000000,0.044118,0.970588,0.029412,1.132353,...,68,542.294118,0.031324,0.030000,0.279412,0.367647,0.161765,1,F,950000
4,9,Aaron Volpatti,1.035088,0.096491,0.043860,0.017544,0.052632,2.254386,0.017544,0.640351,...,114,476.307018,0.020000,0.021404,0.219298,0.280702,0.017544,1,F,950000
5,10,Aaron Voros,0.803738,0.102804,0.102804,0.037383,0.065421,1.710280,0.037383,0.859813,...,107,530.196262,0.014112,0.014019,0.495327,0.186916,0.121495,1,F,4000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,2851,Zachary Sanford,0.965517,0.206897,0.000000,0.000000,0.206897,0.827586,0.000000,0.827586,...,29,650.206897,0.016552,0.016552,0.103448,0.655172,0.206897,1,F,950000
1882,2854,Zack MacEwen,1.407407,0.245370,0.069444,0.009259,0.097222,2.189815,0.023148,1.032407,...,216,569.185185,0.013056,0.012963,0.282407,0.384259,0.074074,1,F,2000000
1883,2855,Zack Mitchell,0.823529,0.029412,0.088235,0.058824,0.147059,1.176471,0.000000,0.647059,...,34,591.323529,0.022647,0.022647,0.058824,0.382353,0.058824,1,F,950000
1884,2856,Zack Ostapchuk,0.857143,0.142857,0.000000,0.000000,0.000000,1.857143,0.000000,0.714286,...,7,629.857143,0.058571,0.054286,0.142857,0.857143,0.000000,1,F,950000


In [25]:
# List of players to keep
players_to_keep = ['Shane Wright', 'Sam Colangelo', 'Frank Nazar']

# Filter out players in over_8000000 except those in players_to_keep
filtered_over_8000000 = over_8000000[~over_8000000['name'].isin(players_to_keep)]

# Remove the filtered players from forwards_rec
forwards_rec = forwards_rec[~forwards_rec['name'].isin(filtered_over_8000000['name'])]


In [28]:
# Define the over_8000000 DataFrame based on the criteria
over_8000000 = forwards_rec[(forwards_rec['salary'] == 4000000) & (forwards_rec['games_played'] < 50)]

# Change the salary of all players in the over_8000000 DataFrame to $950,000 in the forwards_rec DataFrame
forwards_rec.loc[over_8000000.index, 'salary'] = 950000

In [36]:
# Define the over_8000000 DataFrame based on the criteria
over_8000000 = forwards_rec[forwards_rec['games_played'] < 246]

# Change the salary of all players in the over_8000000 DataFrame to $950,000 in the forwards_rec DataFrame
forwards_rec.loc[over_8000000.index, 'salary'] = 950000

In [37]:
forwards_rec.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/forwards_rec_two.csv')